# PCB Defect Detection System
## Automated Optical Inspection (AOI) for Electronics Manufacturing

---

### Industrial Engineering Context

**Problem Statement:**
- Manual PCB inspection is slow (100-200 boards/hour) and error-prone (10-30% false negatives)
- Defective boards reaching customers result in costly recalls and brand damage
- Human inspectors suffer from fatigue, leading to inconsistent quality control

**Objective:**
Build a Deep Learning classifier to automatically identify PCB defects with >95% accuracy, enabling:
- Real-time quality control on production lines
- Consistent defect detection 24/7
- Reduced labor costs and improved throughput
- Prevention of defective products reaching market

**Target Defects:**
- Mouse Bite (incomplete routing)
- Open Circuit (broken traces)
- Short Circuit (unintended connections)
- Spurious Copper (excess material)
- Spur (sharp protrusions)
- Missing Hole

**Technical Approach:**
- Model: MobileNetV2 (lightweight, edge-deployable)
- Dataset: PCB Defects by Akhatova
- Augmentation: Simulate conveyor belt variations
- Class Balancing: Handle imbalanced defect distributions

---

## 1. Environment Setup

In [ ]:
# Install required packages (if not already installed)
import sys

# Check if running on Kaggle
IS_KAGGLE = '/kaggle/input' in sys.path or 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

if not IS_KAGGLE:
    !pip install -q tensorflow numpy pandas matplotlib seaborn scikit-learn Pillow opencv-python

In [ ]:
# Import libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from pathlib import Path

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print(f"Environment: {'Kaggle' if IS_KAGGLE else 'Local'}")

## 2. Add Source Code to Path

For Kaggle: Upload the `src/` folder as a dataset or copy the code directly into cells.
For Local: Ensure you're running from the project root.

In [ ]:
# Add src directory to path
if IS_KAGGLE:
    # If you uploaded src as a dataset, adjust path accordingly
    # sys.path.append('/kaggle/input/pcb-defect-detector-src')
    pass
else:
    # Local environment
    project_root = Path.cwd().parent
    sys.path.insert(0, str(project_root))

print(f"Python Path: {sys.path[:3]}")

## 3. Import Custom Modules

**Note for Kaggle Users:**
If you get import errors, copy the contents of each `.py` file from the `src/` folder into separate cells below.

In [ ]:
# Import custom modules
try:
    from src.config import Config
    from src.data_ingestion import DataIngestion
    from src.model import PCBClassifier
    from src.trainer import TrainingManager
    print("✓ All modules imported successfully")
except ImportError as e:
    print(f"Import Error: {e}")
    print("\nFor Kaggle: Copy the code from src/ files into cells below")
    print("For Local: Ensure you're in the project root directory")

## 4. Verify Dataset Path

In [ ]:
# Check dataset path
data_path = Config.get_data_path()
print(f"Data Path: {data_path}")
print(f"Path Exists: {data_path.exists()}")

if data_path.exists():
    print("\nDataset Structure:")
    for item in sorted(data_path.iterdir())[:10]:
        print(f"  {item.name}")
else:
    print("\n⚠️ Dataset not found!")
    print("For Kaggle: Add 'akhatova/pcb-defects' dataset to your notebook")
    print("For Local: Download dataset using kaggle_setup.py")

## 5. Run Complete Training Pipeline

This single command executes:
1. Data loading and analysis
2. Class imbalance handling
3. Model building (MobileNetV2)
4. Training with callbacks
5. Evaluation and visualization
6. Model export for deployment

In [ ]:
# Initialize training manager
trainer = TrainingManager()

# Run complete pipeline
metrics = trainer.run_pipeline()

## 6. Analyze Results

In [ ]:
# Display final metrics
print("\n" + "="*60)
print("FINAL PERFORMANCE SUMMARY")
print("="*60)
for metric, value in metrics.items():
    print(f"{metric.upper():20s}: {value:.4f}")
print("="*60)

# Check if meets industrial requirements
print("\nIndustrial Requirements Check:")
print(f"  Accuracy > 95%:   {'✓ PASS' if metrics['accuracy'] > 0.95 else '✗ FAIL'}")
print(f"  Precision > 93%:  {'✓ PASS' if metrics['precision'] > 0.93 else '✗ FAIL'}")
print(f"  Recall > 90%:     {'✓ PASS' if metrics['recall'] > 0.90 else '✗ FAIL'}")

## 7. Model Inference Example

In [ ]:
# Load a sample image for inference
from tensorflow.keras.preprocessing import image

# Get a random validation image
val_generator = trainer.data_ingestion.val_generator
val_generator.reset()

# Get one batch
sample_batch, sample_labels = next(val_generator)

# Make predictions
predictions = trainer.model_wrapper.model.predict(sample_batch[:5])

# Visualize predictions
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
class_names = trainer.data_ingestion.class_names

for i in range(5):
    axes[i].imshow(sample_batch[i])
    true_label = class_names[np.argmax(sample_labels[i])]
    pred_label = class_names[np.argmax(predictions[i])]
    confidence = np.max(predictions[i]) * 100
    
    color = 'green' if true_label == pred_label else 'red'
    axes[i].set_title(f"True: {true_label}\nPred: {pred_label}\nConf: {confidence:.1f}%",
                     color=color, fontweight='bold')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 8. Export Model for Production

The model has been saved in multiple formats:
- **Keras (.h5)**: For Python applications
- **SavedModel**: For TensorFlow Serving
- **Architecture JSON**: For documentation

### Deployment Options:
1. **Edge Devices**: Convert to TensorFlow Lite for Raspberry Pi, NVIDIA Jetson
2. **Cloud API**: Deploy with TensorFlow Serving or FastAPI
3. **Factory PLC**: Integrate with industrial control systems
4. **Web Interface**: Build React/Vue dashboard for operators

In [ ]:
# List saved model files
output_path = Config.get_output_path()
print("Saved Model Artifacts:")
for item in output_path.iterdir():
    if item.is_file():
        size_mb = item.stat().st_size / (1024**2)
        print(f"  {item.name:40s} ({size_mb:.2f} MB)")
    elif item.is_dir():
        print(f"  {item.name:40s} (directory)")

## 9. Optional: Fine-Tuning

If initial results don't meet requirements, enable fine-tuning to train some base layers.

In [ ]:
# Uncomment to enable fine-tuning
# trainer.model_wrapper.enable_fine_tuning(num_layers=20, learning_rate=1e-5)
# trainer.train_model()
# trainer.evaluate_model()

---

## Conclusion

This notebook demonstrates a complete industrial-grade PCB defect detection system:

✓ **Problem-Focused**: Addresses real manufacturing challenges

✓ **Production-Ready**: OOP structure, proper error handling, comprehensive logging

✓ **Deployment-Optimized**: Lightweight model suitable for edge devices

✓ **Well-Documented**: Clear explanations of industrial context and technical decisions

✓ **Kaggle-Compatible**: Automatic path detection and environment handling

### Next Steps:
1. Deploy model to production environment
2. Integrate with factory AOI systems
3. Monitor performance and retrain with new defect examples
4. Implement A/B testing against current inspection methods

---